# Optimization of food combinations
This Jupyter Notebook aims at finding optimal combinations of foods given a subset of foods and a subset of nutrients to optimize on. Each nutrient has 3 categories of intake: adequate, moderate- and severe-risk associated. The weighted error is increased when falling in a unhealthy intake (too much or not enough).

For now this algorithm does not work properly. It finds combinations that are suboptimal and cannot maximize the number of nutrient that fall into 'adequate intake'. It seems the algorithm falls in the corners of this 30-dimension 'eggholder' by frequently finding 0 or 2100 kcal for some foods.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import linalg, optimize

In [3]:
# import nutritional values of foods for 100 kcal
nut_calc = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Nutrition Calculator (p100kcal)',
                         skiprows=(0,1,3), index_col=0)
# import nutritional needs for humans
nut_needs = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Human Nutritional Needs',
                          nrows=5, skiprows=(0,2), index_col=0)

In [4]:
nut_calc = nut_calc.fillna(0)

In [5]:
nut_needs

,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,Sugars,Vitamin A,Vitamin E,Vitamin D,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Lower (severe risk),46.00,35.00,1.000000e-99,1.000000e-99,5.000000e+01,2.500000e+01,1.000000e-99,5.000000e+02,1.000000e+04,3.8,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,1.350000e+00,1.000000e+01
Lower (moderate risk),51.00,46.67,1.000000e-99,1.000000e-99,1.300000e+02,2.500000e+01,1.000000e-99,7.000000e+02,1.500000e+04,15.0,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,2.000000e+00,1.450000e+01
Higher (safe),78.75,70.00,2.333000e+01,2.300000e+00,4.000000e+02,7.000000e+01,5.250000e+01,3.000000e+03,3.000000e+05,250.0,...,7.440000e+00,2.480000e+00,9.920000e-01,3.720000e+00,9.920000e-01,3.720000e+00,9.920000e-01,6.448000e+00,2.800000e+00,1.867000e+01
Higher (moderate risk),140.00,81.67,2.333000e+01,2.300000e+00,5.000000e+02,5.000000e+02,1.312500e+02,8.000000e+03,1.000000e+06,1925.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,4.670000e+00,2.333000e+01
Higher (severe risk),400.00,200.00,2.000000e+02,2.000000e+02,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,15000.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99


In [6]:
nut_calc

,Notes,Source,Energy,Protein digestibility,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Food,,,,,,,,,,,,,,,,,,,,,
"Fish, tuna, fresh, skipjack, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,103.000000,0.900,21.359223,0.980583,0.318447,0.000000,0.000000,0.000000,...,1.961165,0.632039,0.229126,0.833981,0.721359,0.935922,0.238835,1.100000,0.000000,0.015534
"Cattle (meat, mechanically separated)","mechanically separated, raw",https://fdc.nal.usda.gov/fdc-app.html#/food-de...,276.000000,0.950,5.423913,8.521739,4.268116,0.000000,0.000000,0.000000,...,0.420652,0.154348,0.083333,0.233333,0.138043,0.171739,0.060870,0.331522,0.061594,0.213768
"Beef, variety meats and by-products, liver, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,135.000000,0.950,15.081481,2.688889,0.913333,0.125926,2.881481,0.000000,...,1.190370,0.402222,0.278519,0.802963,0.597778,0.643704,0.194815,0.933333,0.005185,0.221481
"Chicken, broilers or fryers, meat and skin and giblets and neck, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,213.000000,1.000,8.605634,6.962441,1.990610,0.000000,0.061033,0.000000,...,0.690141,0.226291,0.115493,0.337089,0.275117,0.356808,0.095305,0.419249,0.000000,1.328638
"Mushrooms, shiitake, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,34.000000,0.690,6.588235,1.441176,0.000000,0.000000,19.970588,7.352941,...,0.394118,0.097059,0.064706,0.326471,0.229412,0.394118,0.032353,0.426471,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Beef, tripe, cooked, simmered",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,94.000000,0.971,12.446809,4.308511,1.446809,0.212766,2.117021,0.000000,...,0.724404,0.296856,0.125090,0.413234,0.465511,0.470489,0.118245,0.000000,0.008511,0.145745
Cattle (organs),calculated from individual organ nutritional v...,0,139.808421,0.950,14.722854,3.494903,1.369257,0.144928,1.504420,0.000000,...,1.033903,0.363285,0.127949,0.387572,0.329675,0.606136,0.173915,0.180805,0.012212,0.175925
Lignocellulosic sugar,sugar from cellulose,0,400.000000,0.000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Trying to combine foods to reach avg of adequate intake.
With

`food_subset=['Beef, variety meats and by-products, mechanically separated beef, raw',
             'Milk, whole, 3.25% milkfat, without added vitamin A and vitamin D',
            'Rice, white, long-grain, regular, raw, unenriched',
            'Corn, sweet, yellow, raw','Barley, hulled','Anchovy, canned','Soy flour, full-fat, raw',
            'Soybeans, mature seeds, raw', 'Beef, organ meal, cooked', 'Bread, whole-wheat']
`

We should find something like `[100, 100, 250, 300, 200, 250, 400, 150, 100, 250]`

In [15]:
def f(x, target): # function giving normalized error, target is the middle of the adequate intake range
    return (target-x)/target

def weighted_errors(val, nut_needs_subset, nut, target):
    # get all thresholds
    lower_severe = nut_needs_subset.at['Lower (severe risk)', nut]
    lower_moderate = nut_needs_subset.at['Lower (moderate risk)', nut]
    higher_safe = nut_needs_subset.at['Higher (safe)', nut]
    higher_moderate = nut_needs_subset.at['Higher (moderate risk)', nut]
    higher_severe = nut_needs_subset.at['Higher (severe risk)', nut]
    # put differentiated weight depending on intake category, should be continuous
    if val>higher_severe:
        y = np.exp(f(val, target)) - np.exp(f(higher_severe, target)) + f(higher_severe, target)**4 - f(higher_moderate, target)**4 + f(higher_moderate, target)**2 - f(higher_safe, target)**2
    elif val>higher_moderate:
        y = f(val, target)**4 - f(higher_moderate, target)**4 + f(higher_moderate, target)**2 - f(higher_safe, target)**2
    elif val>higher_safe:
        y = f(val, target)**2 - f(higher_safe, target)**2
    elif (val>= lower_moderate) and (val<=higher_safe):
        y = 0
    elif val<lower_severe:
        y = f(val, target)**4 - f(lower_severe, target)**4 + f(lower_severe, target)**2 - f(lower_moderate, target)**2
    elif val<lower_moderate:
        y = f(val, target)**2 - f(lower_moderate, target)**2
    return y


def intake_cat(X, nut_needs, nut_subset, nut_calc, food_subset):# gives the sum of weighted errors for each nutrient
    nut_calc_subset=nut_calc.loc[food_subset]
    nut_calc_subset=np.array(nut_calc_subset[nut_subset])
    nut_needs_subset = nut_needs[nut_subset]
    #target = 1.1*np.array(nut_needs_subset.loc['Lower (moderate risk)']
    target = (np.array(nut_needs_subset.loc['Lower (moderate risk)'])+np.array(nut_needs_subset.loc['Higher (safe)']))/2
    solution = np.abs(np.dot(np.transpose(nut_calc_subset), X)-target)
    Y=np.zeros((len(nut_subset)))
    for i in range(len(nut_subset)):
        nut = nut_subset[i]
        val = solution[i]
        Y[i] = weighted_errors(val, nut_needs_subset, nut, target[i])
    return np.sum(Y)

In [16]:
diets_excel=pd.ExcelFile("Diets by period.xlsx")

sheets = diets_excel.sheet_names
diets = pd.read_excel("Diets by period.xlsx", index_col=0, sheet_name=sheets[0])
# select some foods to combine
diets_bis = diets.drop('Total:')
food_subset = list(diets_bis.index)

# select the nutrients we want to look at (bug when lower bound is 0 so those nutrients are removed)
nut_subset=['Protein', 'Fat total',
            #'Saturated fats', 'Trans fats',
            'Carbohydrates','Fiber',
            #'Sugars',
            'Vitamin A', 'Vitamin E', 'Vitamin D', 'Vitamin C',
           'Thiamine (B1)', 'Riboflavin (B2)', 'Niacin (B3)', 'Vitamin B6',
           'Vitamin B12', 'Vitamin K', 'Folate (B9)', 'Pantothenic Acid (B5)',
           'Calcium', 'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium',
           'Zinc', 'Copper', 'Manganese', 'Selenium', 'Iodine', 'Omega-3 (ALA)', 'Omega-6 (LA)']


In [17]:
bounds = optimize.Bounds(np.zeros(len(food_subset)), 21*np.ones(len(food_subset))) # bounds are 0 and 21*100 kcal for each food
constraint = optimize.LinearConstraint(np.ones(len(food_subset)), 21, 21) # the sum of all caloric intake should be 21(*100 kcal)

coef_list = []
n_trials=10
for i in range(n_trials):
    X0 = 10*np.random.rand(len(food_subset), 1) # start with a somewhat random initial guess
    coefs = optimize.minimize(intake_cat, X0,
                              (nut_needs, nut_subset, nut_calc, food_subset),
                              constraints=constraint,
                              bounds=bounds,
                             method='SLSQP') # optimize using SLSQP method
    if coefs.success: # gather results to find the min of several trials
        coef_list += [[coefs.x, coefs.fun]]
        print(i)
        for j in range(len(food_subset)):
            print(food_subset[j][:12], '...:', round(coefs.x[j]*100), 'kcal')
# find the min of the different results
coef_array = np.array(coef_list)
i_min = np.argmin(coef_array[:, 1])
selected_coef = coef_array[i_min][0]
print('Min of', n_trials, 'trials') # print the resulting diet
for i in range(len(food_subset)):
    print(food_subset[i][:12], '...:', round(selected_coef[i]*100), 'kcal')

0
Cattle (meat ...: 224 kcal
Milk (whole) ...: 0 kcal
Rice ...: 1264 kcal
Potatoes ...: 140 kcal
Barley ...: 0 kcal
Anchovy ...: 0 kcal
Canola oil ...: 472 kcal
Soy flour ...: 0 kcal
Cattle (orga ...: 0 kcal
Bread (whole ...: 0 kcal
1
Cattle (meat ...: 1 kcal
Milk (whole) ...: 94 kcal
Rice ...: 1881 kcal
Potatoes ...: 97 kcal
Barley ...: 0 kcal
Anchovy ...: 0 kcal
Canola oil ...: 26 kcal
Soy flour ...: 0 kcal
Cattle (orga ...: 0 kcal
Bread (whole ...: 0 kcal
2
Cattle (meat ...: 2 kcal
Milk (whole) ...: 92 kcal
Rice ...: 1883 kcal
Potatoes ...: 96 kcal
Barley ...: 0 kcal
Anchovy ...: 0 kcal
Canola oil ...: 27 kcal
Soy flour ...: 0 kcal
Cattle (orga ...: 0 kcal
Bread (whole ...: 0 kcal
3
Cattle (meat ...: 0 kcal
Milk (whole) ...: 0 kcal
Rice ...: 1605 kcal
Potatoes ...: 0 kcal
Barley ...: 0 kcal
Anchovy ...: 495 kcal
Canola oil ...: 0 kcal
Soy flour ...: 0 kcal
Cattle (orga ...: 0 kcal
Bread (whole ...: 0 kcal
4
Cattle (meat ...: 547 kcal
Milk (whole) ...: 0 kcal
Rice ...: 1068 kcal
Pota

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [18]:
nut_calc_subset=nut_calc.loc[food_subset]
nut_calc_subset_t=nut_calc_subset[nut_needs.columns].T
combination = pd.Series(selected_coef, index=food_subset)
#combination = pd.Series([100, 100, 250, 300, 200, 250, 400, 150, 100, 250], index=food_subset)/100
diet = nut_calc_subset_t @ combination
print(diet)

nut_needs_subset = nut_needs[nut_subset]
SRAI = []
MRAI = []
AI = []
danger = []
for i in range(len(diet)): # for the solution, divide in categories for each nutrient
    nutrient=diet[i]
    nut=diet.index[i]
    lower_severe = nut_needs.at['Lower (severe risk)', nut]
    lower_moderate = nut_needs.at['Lower (moderate risk)', nut]
    higher_safe = nut_needs.at['Higher (safe)', nut]
    higher_moderate = nut_needs.at['Higher (moderate risk)', nut]
    higher_severe = nut_needs.at['Higher (severe risk)', nut]
    if (nutrient>= lower_moderate) and (nutrient<=higher_safe):
        AI += [nut]
    elif (nutrient==0):
        danger+= [nut + ' 0%']
    elif (nutrient>higher_severe):
        danger+= [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
    elif (nutrient<lower_severe):
        SRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
    elif (nutrient>higher_moderate):
        SRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
    elif (nutrient< lower_moderate):
        MRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
    elif (nutrient>higher_safe):
        MRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
        
print('AI', AI, '\n\nMRAI', MRAI, '\n\nSRAI', SRAI, '\n\nDanger', danger)

Protein                    43.771276
Fat total                  11.664714
Saturated fats              4.117473
Trans fats                  0.011768
Carbohydrates             441.845556
Fiber                       8.705062
Sugars                      9.426902
Vitamin A                  71.001442
Vitamin E                1206.273879
Vitamin D                   0.154351
Vitamin C                  14.481824
Thiamine (B1)               0.550231
Riboflavin (B2)             0.536296
Niacin (B3)                 9.999979
Vitamin B6                  1.235410
Vitamin B12                 0.708220
Vitamin K                   5.553395
Folate (B9)                60.114126
Pantothenic Acid (B5)       6.382400
Calcium                   326.875972
Iron                        4.544859
Magnesium                 168.877353
Phosphorus                773.078850
Potassium                1220.102346
Sodium                     96.899364
Zinc                        6.541967
Copper                   1382.142556
M

In [ ]:
# just to see the shape of the weighted error function, that should be continuous
nut_subset_fig=['Fat total']

nut_calc_subset=nut_calc.loc[food_subset]
nut_calc_subset=nut_calc_subset[nut_subset_fig]
nut_needs_subset = nut_needs[nut_subset_fig]
nut = nut_needs_subset.columns[0]

lower_severe = nut_needs_subset.at['Lower (severe risk)', nut]
lower_moderate = nut_needs_subset.at['Lower (moderate risk)', nut]
higher_safe = nut_needs_subset.at['Higher (safe)', nut]
higher_moderate = nut_needs_subset.at['Higher (moderate risk)', nut]
higher_severe = nut_needs_subset.at['Higher (severe risk)', nut]

target = (np.array(nut_needs_subset.loc['Lower (moderate risk)'])+np.array(nut_needs_subset.loc['Higher (safe)']))/2

X = np.linspace(0, higher_moderate+10)
Y=np.zeros(X.shape)

for i in range(len(X)):
    val=X[i]
    Y[i] = weighted_errors(val, nut_needs_subset, nut, target)   
    #print(val, err, Y[i])
        
plt.plot(X, Y)
plt.axvline(0, color='r')
plt.axvline(lower_severe, color='orange')
plt.axvline(lower_moderate, color='green')
plt.axvline(higher_safe, color='lightgreen')
plt.axvline(higher_moderate, color='peachpuff')
#plt.axvline(higher_severe, color='lightcoral')
plt.title(nut_subset_fig[0] + ' weighted error function')

## A la mano
Caloric intake determined by hand, calculation of the weighted error function and adjustements when possible

In [ ]:
diets_excel=pd.ExcelFile("Diets by period.xlsx")

sheets = diets_excel.sheet_names
diets = pd.read_excel("Diets by period.xlsx", index_col=0, sheet_name=sheets[0])

In [ ]:
diets = diets.drop('Total:')
diets

In [ ]:
X = diets['kcal']
X

In [ ]:
nut_subset=['Protein', 'Fat total',
            'Saturated fats', 'Trans fats',
            'Carbohydrates','Fiber',
            'Sugars',
            'Vitamin A', 'Vitamin E', 'Vitamin D', 'Vitamin C',
           'Thiamine (B1)', 'Riboflavin (B2)', 'Niacin (B3)', 'Vitamin B6',
           'Vitamin B12', 'Vitamin K', 'Folate (B9)', 'Pantothenic Acid (B5)',
           'Calcium', 'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium',
           'Zinc', 'Copper', 'Manganese', 'Selenium', 'Iodine', 'Omega-3 (ALA)', 'Omega-6 (LA)']
food_subset = list(diets.index)
y1 = intake_cat(X, nut_needs, nut_subset, nut_calc, food_subset)

In [ ]:
X = [400, 300, 650, 250, 250, 250]
y2 = intake_cat(X, nut_needs, nut_subset, nut_calc, food_subset)
print(np.sum(X), y2-y1)

In [ ]:
y1=y2
X = [500, 300, 650, 250, 250, 150]
y2 = intake_cat(X, nut_needs, nut_subset, nut_calc, food_subset)
print(np.sum(X), y2-y1)